# Problem 2

## (a)

In [153]:
import pandas as pd

df = pd.read_excel('ccapm.xlsx', names=['cratio', 'rrate', 'e'], header=None)

In [154]:
df

,cratio,rrate,e
0,1.010612,1.006276,0.034638
1,0.991563,1.035976,0.106118
2,1.009609,1.018062,-0.084368
3,0.999425,0.996259,0.096085
4,0.996430,1.034679,-0.005752
...,...,...,...
233,1.006409,1.031475,0.025306
234,1.004874,0.985399,0.064093
235,0.999018,0.890846,0.048739
236,1.002982,1.024748,-0.009818


In [155]:
from statsmodels.sandbox.regression.gmm import GMM
import numpy as np

# i think the ode does not use endog, exog, intruments so I have just put random things here to get it to not throw an erropr
# this is a terrible package.
cd = np.array(range(len(df)))
dcd = np.array(range(len(df)))
inst = np.column_stack((np.ones(len(cd)), cd, cd, cd))

class Model(GMM):
    def momcond(self, params):
        cratio, rrate, e = df['cratio'], df['rrate'], df['e']
        beta, gamma = params

        m1 = beta * cratio**(-gamma) * rrate - 1
        m2 = (beta * cratio**(-gamma) * rrate - 1) * cratio.shift(1)
        m3 = (beta * cratio**(-gamma) * rrate - 1) * rrate.shift(1)
        m4 = (beta * cratio**(-gamma) * rrate - 1) * e.shift(1)

        # drop NaNs from all series simultaneously
        M = pd.concat([m1, m2, m3, m4], axis=1).dropna()

        return M.to_numpy()

n = len(df)
y = np.zeros(n)
X = np.ones((n,1))
Z = np.ones((n,1))

model = Model(endog = dcd, exog = cd, instrument = inst, k_mom=4, k_params=2)
res=model.fit(start_params=[1, 1])

Optimization terminated successfully.
         Current function value: 0.000000
         Iterations: 1
         Function evaluations: 3
         Gradient evaluations: 3
Optimization terminated successfully.
         Current function value: 0.005432
         Iterations: 5
         Function evaluations: 10
         Gradient evaluations: 10
Optimization terminated successfully.
         Current function value: 0.005188
         Iterations: 4
         Function evaluations: 9
         Gradient evaluations: 9
Optimization terminated successfully.
         Current function value: 0.005197
         Iterations: 1
         Function evaluations: 3
         Gradient evaluations: 3
Optimization terminated successfully.
         Current function value: 0.005196
         Iterations: 1
         Function evaluations: 3
         Gradient evaluations: 3


In [156]:
print(res.summary())

                                Model Results                                 
Dep. Variable:                      y   Hansen J:                        1.231
Model:                          Model   Prob (Hansen J):                 0.540
Method:                           GMM                                         
Date:                Wed, 03 Dec 2025                                         
Time:                        18:39:31                                         
No. Observations:                 238                                         
                 coef    std err          z      P>|z|      [0.025      0.975]
------------------------------------------------------------------------------
p 0            0.9978      0.004    230.494      0.000       0.989       1.006
p 1            0.7723      1.770      0.436      0.663      -2.698       4.242


## (b)

In [157]:
model = Model(endog = dcd, exog = cd, instrument = inst, k_mom=4, k_params=2)
res=model.fit(start_params=[1, 1], weights_method='hac', wargs={"maxlag": 5})
print(res.summary())

Optimization terminated successfully.
         Current function value: 0.000000
         Iterations: 1
         Function evaluations: 3
         Gradient evaluations: 3
Optimization terminated successfully.
         Current function value: 0.008025
         Iterations: 5
         Function evaluations: 10
         Gradient evaluations: 10
Optimization terminated successfully.
         Current function value: 0.007271
         Iterations: 4
         Function evaluations: 9
         Gradient evaluations: 9
Optimization terminated successfully.
         Current function value: 0.007278
         Iterations: 1
         Function evaluations: 3
         Gradient evaluations: 3
Optimization terminated successfully.
         Current function value: 0.007277
         Iterations: 1
         Function evaluations: 3
         Gradient evaluations: 3
                                Model Results                                 
Dep. Variable:                      y   Hansen J:                        1

In [158]:
tstat = (res.params[0] - 0.98) / res.bse[0]

from scipy import stats
p_value = 2 * (1 - stats.norm.cdf(abs(tstat)))
print("P-value:", p_value)
print("T-statistic:", tstat)

P-value: 4.6688968315722335e-05
T-statistic: 4.071611677371312


In [159]:
float(4.6688968315722335e-05)

4.6688968315722335e-05

In [ ]:
class Model(GMM):
    def calc_weightmatrix(self, moms, *args, **kwargs):
        """
        moms : (nobs x nmoms) array
        lags : integer, number of autocovariance lags G
        """
        lags = 5

        n, m = moms.shape
        Gamma = np.zeros((lags+1, m, m))

        # gamma
        for j in range(lags+1):
            prod = moms[j:]   # g_i
            lagged = moms[:n-j]  # g_{i-j}
            Gamma[j] = (prod.T @ lagged) / n

        # weights
        V = Gamma[0].copy()
        for j in range(1, lags+1):
            weight = (lags + 1 - j) / (lags + 1)
            V += weight * (Gamma[j] + Gamma[j].T)

        return V
    
    def momcond(self, params):
        cratio, rrate, e = df['cratio'], df['rrate'], df['e']
        beta, gamma = params

        m1 = beta * cratio**(-gamma) * rrate - 1
        m2 = (beta * cratio**(-gamma) * rrate - 1) * cratio.shift(1)
        m3 = (beta * cratio**(-gamma) * rrate - 1) * rrate.shift(1)
        m4 = (beta * cratio**(-gamma) * rrate - 1) * e.shift(1)

        M = pd.concat([m1, m2, m3, m4], axis=1).dropna()

        return M.to_numpy()

model = Model(endog = dcd, exog = cd, instrument = inst, k_mom=4, k_params=2)
res=model.fit(start_params=[1, 1])
print(res.summary())

Optimization terminated successfully.
         Current function value: 0.000000
         Iterations: 1
         Function evaluations: 3
         Gradient evaluations: 3
Optimization terminated successfully.
         Current function value: 0.008005
         Iterations: 5
         Function evaluations: 10
         Gradient evaluations: 10
Optimization terminated successfully.
         Current function value: 0.007896
         Iterations: 2
         Function evaluations: 7
         Gradient evaluations: 7
Optimization terminated successfully.
         Current function value: 0.007888
         Iterations: 1
         Function evaluations: 3
         Gradient evaluations: 3
Optimization terminated successfully.
         Current function value: 0.007887
         Iterations: 2
         Function evaluations: 7
         Gradient evaluations: 7
Optimization terminated successfully.
         Current function value: 0.007887
         Iterations: 1
         Function evaluations: 3
         Gradient

In [161]:
tstat = (res.params[0] - 0.98) / res.bse[0]

p_value = 2 * (1 - stats.norm.cdf(abs(tstat)))
print("P-value:", p_value)
print("T-statistic:", tstat)

P-value: 5.303290473412936e-05
T-statistic: 4.041842256927658


# Problem 4

## (a)

In [162]:
df = pd.read_table('minwage.txt')

In [163]:
df

,chain,own,state,empft,emppt,nmgrs,wagest,empft2,emppt2,nmgrs2,wagest2
0,1,0,0,30,15,3,.,3.5,35,3,4.3
1,2,0,0,6.5,6.5,4,.,0,15,4,4.45
2,2,1,0,3,7,2,.,3,7,4,5
3,4,1,0,20,20,4,5,0,36,2,5.25
4,4,1,0,6,26,5,5.5,28,3,6,4.75
...,...,...,...,...,...,...,...,...,...,...,...
405,2,1,1,2,10,2,4.95,19,3.5,3,5.25
406,2,1,1,3.5,6.5,3,4.75,12,3,4,5.25
407,3,1,1,3,33,5,4.25,5,25,3,5.05
408,4,0,1,7,8,3,4.75,0,35,3,5.05


In [164]:
df['empft'] = pd.to_numeric(df['empft'], errors='coerce')
df['nmgrs'] = pd.to_numeric(df['nmgrs'], errors='coerce')
df['emppt'] = pd.to_numeric(df['emppt'], errors='coerce')

df['empft2'] = pd.to_numeric(df['empft2'], errors='coerce')
df['nmgrs2'] = pd.to_numeric(df['nmgrs2'], errors='coerce')
df['emppt2'] = pd.to_numeric(df['emppt2'], errors='coerce')

df['wagest'] = pd.to_numeric(df['wagest'], errors='coerce')
df['wagest2'] = pd.to_numeric(df['wagest2'], errors='coerce')

df.dropna(subset=['empft', 'nmgrs', 'emppt', 'empft2', 'nmgrs2', 
                  'emppt2', 'wagest', 'wagest2'], inplace=True)

In [165]:
len(df)

351

In [166]:
df['fte'] = df['empft'] + df['nmgrs'] + 0.5 * df['emppt']
df['fte2'] = df['empft2'] + df['nmgrs2'] + 0.5 * df['emppt2']

In [167]:
df.groupby('state')[['fte', 'fte2']].mean()

,fte,fte2
state,,
0,23.704545,21.825758
1,20.678246,21.076316


## (b)

In [168]:
# Calculate the change in employment
df['dfte'] = df['fte2'] - df['fte']

# Create treatment variable (New Jersey = 1, Pennsylvania = 0)
df['treat'] = df['state']

# Run difference-in-differences regression
import statsmodels.api as sm

# Add constant for intercept
X_did = sm.add_constant(df['treat'])
y_did = df['dfte']

# Fit the regression
did_model = sm.OLS(y_did, X_did).fit()
print(did_model.summary())

                            OLS Regression Results                            
Dep. Variable:                   dfte   R-squared:                       0.010
Model:                            OLS   Adj. R-squared:                  0.008
Method:                 Least Squares   F-statistic:                     3.658
Date:                Wed, 03 Dec 2025   Prob (F-statistic):             0.0566
Time:                        18:39:31   Log-Likelihood:                -1257.0
No. Observations:                 351   AIC:                             2518.
Df Residuals:                     349   BIC:                             2526.
Df Model:                           1                                         
Covariance Type:            nonrobust                                         
                 coef    std err          t      P>|t|      [0.025      0.975]
------------------------------------------------------------------------------
const         -1.8788      1.073     -1.751      0.0

## (c)

In [169]:
df[['chain', 'own']]

,chain,own
3,4,1
4,4,1
6,1,0
7,1,0
8,2,1
...,...,...
405,2,1
406,2,1
407,3,1
408,4,0


In [170]:
did_model_controls = sm.OLS.from_formula(
    "y_did ~ treat + C(chain) + C(own)", data=df
).fit()
print(did_model_controls.summary())

                            OLS Regression Results                            
Dep. Variable:                  y_did   R-squared:                       0.020
Model:                            OLS   Adj. R-squared:                  0.006
Method:                 Least Squares   F-statistic:                     1.436
Date:                Wed, 03 Dec 2025   Prob (F-statistic):              0.211
Time:                        18:39:31   Log-Likelihood:                -1255.2
No. Observations:                 351   AIC:                             2522.
Df Residuals:                     345   BIC:                             2546.
Df Model:                           5                                         
Covariance Type:            nonrobust                                         
                    coef    std err          t      P>|t|      [0.025      0.975]
---------------------------------------------------------------------------------
Intercept        -1.4500      1.210     -1.198

In [ ]:
df_long = pd.melt(
    df, 
    id_vars=['chain', 'own', 'state', 'treat'], 
    value_vars=['fte', 'fte2'],
    var_name='period', 
    value_name='fte_outcome'
)
df_long['post'] = (df_long['period'] == 'fte2').astype(int)

did_explicit = sm.OLS.from_formula(
    "fte_outcome ~ treat * post + C(chain) + C(own)", data=df_long
).fit()
print(did_explicit.summary())

                            OLS Regression Results                            
Dep. Variable:            fte_outcome   R-squared:                       0.218
Model:                            OLS   Adj. R-squared:                  0.210
Method:                 Least Squares   F-statistic:                     27.70
Date:                Wed, 03 Dec 2025   Prob (F-statistic):           1.16e-33
Time:                        18:43:22   Log-Likelihood:                -2478.3
No. Observations:                 702   AIC:                             4973.
Df Residuals:                     694   BIC:                             5009.
Df Model:                           7                                         
Covariance Type:            nonrobust                                         
                    coef    std err          t      P>|t|      [0.025      0.975]
---------------------------------------------------------------------------------
Intercept        26.3896      1.090     24.220

## (d)

In [ ]:
df['gap'] = np.where(
    df['state'] == 1,
    np.maximum(0, 5.05/df['wagest'] - 1),
    0
)

In [ ]:
X_gap = sm.add_constant(df['gap'])
y = df['dfte']
gap_model = sm.OLS(y, X_gap).fit()
print(gap_model.summary())

coef_gap = gap_model.params.get('gap', gap_model.params[1])
mean_gap_NJ = df.loc[df['state'] == 1, 'gap'].mean()

print("mean_gap_NJ * coef_gap =", mean_gap_NJ * coef_gap)

                            OLS Regression Results                            
Dep. Variable:                   dfte   R-squared:                       0.022
Model:                            OLS   Adj. R-squared:                  0.019
Method:                 Least Squares   F-statistic:                     7.839
Date:                Wed, 03 Dec 2025   Prob (F-statistic):            0.00540
Time:                        18:14:42   Log-Likelihood:                -1254.9
No. Observations:                 351   AIC:                             2514.
Df Residuals:                     349   BIC:                             2522.
Df Model:                           1                                         
Covariance Type:            nonrobust                                         
                 coef    std err          t      P>|t|      [0.025      0.975]
------------------------------------------------------------------------------
const         -1.4777      0.694     -2.130      0.0

/var/folders/sg/s_76cq1j0dgfnzdx33p_3kzc0000gn/T/ipykernel_4929/1736462201.py:6: FutureWarning: Series.__getitem__ treating keys as positions is deprecated. In a future version, integer keys will always be treated as labels (consistent with DataFrame behavior). To access a value by position, use `ser.iloc[pos]`
  coef_gap = gap_model.params.get('gap', gap_model.params[1])


## (e)

In [ ]:
model_with_controls = sm.OLS.from_formula(
    "dfte ~ gap + C(state) + C(chain) + C(own)", data=df
).fit()
print(model_with_controls.summary())

                            OLS Regression Results                            
Dep. Variable:                   dfte   R-squared:                       0.030
Model:                            OLS   Adj. R-squared:                  0.013
Method:                 Least Squares   F-statistic:                     1.797
Date:                Wed, 03 Dec 2025   Prob (F-statistic):             0.0988
Time:                        18:14:42   Log-Likelihood:                -1253.4
No. Observations:                 351   AIC:                             2521.
Df Residuals:                     344   BIC:                             2548.
Df Model:                           6                                         
Covariance Type:            nonrobust                                         
                    coef    std err          t      P>|t|      [0.025      0.975]
---------------------------------------------------------------------------------
Intercept        -1.6635      1.211     -1.374